In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
df = spark.read.format("parquet").load("abfss://bronze@ssdatalakeeteproject.dfs.core.windows.net/products")
df.display()

In [0]:
df = df.drop("_rescued_data")
df.display()

In [0]:
df.createOrReplaceTempView("Tem_Products")

In [0]:
%sql
create or replace function databricks_catalog.bronze.discount_func(p_price double)
returns DOUBLE
LANGUAGE SQL
return p_price * 0.90;


In [0]:
%sql
select product_id, price, databricks_catalog.bronze.discount_func(price) as discounted_price
from Tem_Products


In [0]:
df = df.withColumn("discounted_price",expr("databricks_catalog.bronze.discount_func(price)"))
df.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_catalog.bronze.upper_func(p_brnad STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$

    return p_brnad.upper()
$$

In [0]:
%sql
select product_id, brand, databricks_catalog.bronze.upper_func(brand) as brand_upper
from Tem_Products

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@ssdatalakeeteproject.dfs.core.windows.net/products")

In [0]:
%sql
create table if not exists databricks_catalog.silver.products_silver
using delta
location "abfss://silver@ssdatalakeeteproject.dfs.core.windows.net/products"

Functions
